# Crash Course databases and python (and some xml)

Parts of the examples are from http://www.dr-chuck.com/)

Structured Query Language(SQL)  is the database language by the use of which we can perform certain operations on the existing database and also we can use this language to create a database. SQL uses certain commands like Create, Drop, Insert etc. to carry out the required tasks.

These SQL commands are mainly categorized into four categories as

- Data definition language (DDL): `CREATE`, `DROP`, `ALTER`, `ADD`, `CHECK`
- Data Manipulation language (DML): `SELECT`, `INSERT INTO`, `UPDATE`, `DELETE`, `SET`
- Data Authorization language (DAL): `GRANT`, `REVOKE`
- Data control language (DCL):`CREATE INDEX`

See also https://www.w3schools.com/SQL/

Talking to a database is like making a phonecall. Before we can talk to the database we need to make a connection first. We can connect to a database with the Python Database API. Access to the database is made available through connection objects. The constructor is:

    connect( parameters... )
    
The parameters are database system dependable. In the example below we connect to an sqlite database. 
Sqlite is a free downloadable database system. 

In [17]:
import sqlite3
conn = sqlite3.connect('track.sqlite') #opens or create database

The connection object has several methods: `.close()`, `.commit`, `.rollback`, `.cursor()` where the `cursor()` method returns a new cursor object. This object is mostly used to give 'instructions' to the database. Simple instructions we can give with the `conn.cursor().execute`, multiline scripts we can execute with `conn.cursor().executescript` In the example below a cursor is created and a sql command `DROP TABLE` is given. 

In [18]:
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS Artist;')

With the data definition language we can create tables structures. Good practice is to write a create script for all the tables and executing the whole script with the `conn.cursor().executescript` method

In [19]:
cur.executescript('''
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Track;
DROP TABLE IF EXISTS Genre;


CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Genre (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);

CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    genre_id  INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
);
''')

With the data manipulation language we can fill the table structures

In [20]:
artist = 'Opeth'
cur.execute('INSERT OR IGNORE INTO Artist (name) VALUES ( ? )', ( artist, ) )

And with the data manipulation language we can retrieve information

In [21]:
cur.execute('SELECT * FROM Artist')
print(cur.fetchall())



[(1, 'Opeth')]


To retrieve information from multiple tables into one view we should make `JOIN` statements

### Pandas and SQL

Pandas has a `read_sql_query` function that can read tables from querys. It needs the SQL query in the data manipulation language and a connection. 

In [22]:
import pandas as pd
df = pd.read_sql_query('SELECT * FROM Artist', conn)
print(type(df))
df

<class 'pandas.core.frame.DataFrame'>


,id,name
0,1,Opeth


When we are finised with the database instructions we can close the connection

In [23]:
conn.commit()
conn.close()

## Retrieve from XML

XML and HTML are structured simular but XML is more general. An example of XML you find below. XML is another common structured data format supporting hierarchal nested data with metadata.

In [24]:
import xml.etree.ElementTree as ET

data = '''
<person>
  <name>Fenna</name>
  <phone type="intl">
    +31646080034
  </phone>
  <email hide="yes" />
</person>'''

tree = ET.fromstring(data)
print('Name:', tree.find('name').text)
print('Attr:', tree.find('email').get('hide'))

Name: Fenna
Attr: yes


Unfortunately Pandas package does not have a function to import data from XML so we need to use standard XML package and do some extra work to convert the data to Pandas DataFrames. Below you find an XML file that has track information

In [25]:
fname = 'data/Library.xml'

# <key>Track ID</key><integer>369</integer>
# <key>Name</key><string>Another One Bites The Dust</string>
# <key>Artist</key><string>Queen</string>
def lookup(d, key):
    found = False
    for child in d:
        if found : return child.text
        if child.tag == 'key' and child.text == key :
            found = True
    return None

stuff = ET.parse(fname)
all = stuff.findall('dict/dict/dict') # find third dict which is track
print('Dict count:', len(all))
for entry in all:
    if ( lookup(entry, 'Track ID') is not None ) : 
        name = lookup(entry, 'Name') #title of track
        artist = lookup(entry, 'Artist') # name of artist
        album = lookup(entry, 'Album') #title of album
    print(name, artist, album)

Dict count: 404
Another One Bites The Dust Queen Greatest Hits
Asche Zu Asche Rammstein Herzeleid
Beauty School Dropout Various Grease
Black Dog Led Zeppelin IV
Bring The Boys Back Home Pink Floyd The Wall [Disc 2]
Circles Bryan Lee Blues Is
Comfortably Numb Pink Floyd The Wall [Disc 2]
Crazy Little Thing Called Love Queen Greatest Hits
Electric Funeral Black Sabbath Paranoid
Fat Bottomed Girls Queen Greatest Hits
For Those About To Rock (We Salute You) AC/DC Who Made Who
Four Sticks Led Zeppelin IV
Furious Angels Rob Dougan The Matrix Reloaded
Gelle Bryan Lee Blues Is
Going To California Led Zeppelin IV
Gotta Move Fast Michael Loceff None
Grease Various Grease
Hand of Doom Black Sabbath Paranoid
Hells Bells AC/DC Who Made Who
Hey You Pink Floyd The Wall [Disc 2]
I Worry Bryan Lee Blues Is
Iron Man Black Sabbath Paranoid
Is There Anybody Out There? Pink Floyd The Wall [Disc 2]
It was a Very Good Year Frank Sinatra Greatest Hits
Its Your Move Bryan Lee Blues Is
Jack the Stripper/Fairies

## Exercise 1

Look into the Library.xml file. All the information to fill the track database is in the xml file. Retrieve the information from the xml file and fill all the table structures. Hint: if you need enter an artist in the artist table and you want the id to use as a foreign key to the track table you can use the following code:

In [35]:
conn = sqlite3.connect('track.sqlite')
cur = conn.cursor()

tree = ET.parse(fname)
print(tree.getroot().tag)
print(tree.getroot().attrib)
# print(tree.getroot().attrib.iteritems())
# print(all[0].iterchildren())
print(all[0].tag())



# cur.execute('SELECT id FROM Artist WHERE name = ? ', (artist, ))
# artist_id = cur.fetchone()[0]

plist
{'version': '1.0'}


TypeError: 'str' object is not callable

Check with https://sqlitebrowser.org/ if you succeeded. Make sure that you commit to confirm the transactions

## Exercise 2

In the following url you find an xml file with plant information. Try to fetch the data and retrieve the information. Put it in a pandas dataframe. Remember to decode the data if you retrieve it directly from the web.

In [ ]:
url = 'https://bioinf.nl/~fennaf/DSLS/plants.xml'